In [94]:
import os
import re
import sqlite3 as sql
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from spacy.lang.en import English
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Doc
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from ast import literal_eval
import scipy

import gensim
import nltk
import pyLDAvis.gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

np.random.seed(123)
nltk.download('stopwords')
nltk.download('wordnet')

import warnings
warnings.filterwarnings('ignore')
np.random.seed(123)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gcdunn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/gcdunn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
eng_tweets = pd.read_pickle('../data/eng_tweets.p')

In [114]:
eng_tweets.head(10)

,external_author_id,author,content,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,hashtags,mentions
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,[],[@nedryun]
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,[],[]
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,[#BoycottNFL],[]
3,906000000000000000,10_GOP,JUST IN President Trump dedicates Presidents C...,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,[],[]
4,906000000000000000,10_GOP,19000 RESPECTING our National Anthem,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,[#StandForOurAnthem],[]
5,906000000000000000,10_GOP,"Dan Bongino ""Nobody trolls liberals better tha...",English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,NaN,Right,0,RightTroll,0,905874659358453760,914320835325853696,[],[]
6,906000000000000000,10_GOP,,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,RETWEET,Right,1,RightTroll,0,905874659358453760,914321156466933760,[],[]
7,906000000000000000,10_GOP,' Doesn't matter that CNN doesn't report on yo...,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,NaN,Right,0,RightTroll,0,905874659358453760,914322215537119234,[],"[@SenatorMenendez, @CarmenYulinCruz]"
8,906000000000000000,10_GOP,As much as I hate promoting CNN article here t...,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,NaN,Right,0,RightTroll,0,905874659358453760,914335818503933957,[],[]
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayo...,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,NaN,Right,0,RightTroll,0,905874659358453760,914336862730375170,[],[@CNN]


In [115]:
stemmer = SnowballStemmer('english')

def lemma_stem(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemma_stem(token))
    return result

In [116]:
#eng_tweets['content'] = eng_tweets.apply(lambda row: preprocess(row['content'].lower()), axis=1)

In [117]:
user_documents = eng_tweets.groupby('author',as_index=False).agg({'account_type': list, 'content': sum})
user_documents.head()

,author,account_type,content
0,10_GOP,"[Right, Right, Right, Right, Right, Right, Rig...","""We have a sitting Democrat US Senator on tria..."
1,1D_NICOLE_,"[Koch, Koch, Koch, Koch, Koch, Koch, Koch, Koc...",is not a joke Thanks God that i ate few days ...
2,1ERIK_LEE,"[Right, Right]",Why is someone even against the I'll watch you...
3,1LORENAFAVA1,"[Italian, Italian, Italian, Italian, Italian, ...",Come vedere Juventus-Milan in streaming o in t...
4,2NDHALFONION,"[Right, Right, Right]",' Follow the other half an onion Mage America ...


In [118]:
user_documents.account_type = user_documents.account_type.apply(lambda x: x[0])
user_documents.head()

,author,account_type,content
0,10_GOP,Right,"""We have a sitting Democrat US Senator on tria..."
1,1D_NICOLE_,Koch,is not a joke Thanks God that i ate few days ...
2,1ERIK_LEE,Right,Why is someone even against the I'll watch you...
3,1LORENAFAVA1,Italian,Come vedere Juventus-Milan in streaming o in t...
4,2NDHALFONION,Right,' Follow the other half an onion Mage America ...


In [119]:
user_documents.account_type.value_counts()

Right         593
Russian       551
Left          216
?             190
German        117
Koch          115
Hashtager     106
local          36
Arabic         22
news           15
French          9
Italian         8
Commercial      6
Ebola           2
Spanish         2
ZAPOROSHIA      1
Name: account_type, dtype: int64

In [120]:
user_documents[user_documents.account_type=='local'].head()

,author,account_type,content
77,ALBUQUERQUEON,local,Albuquerque men arrested for trying to sell st...
246,ATLANTA_ONLINE,local,Women Accused of Stealing in Front of Kids \| ...
264,BALTIMORE0NLINE,local,Istanbul Governor At Least 35 Dead In Attack O...
277,BATONROUGEVOICE,local,Jrue Holiday no longer on minute restrictions ...
408,CAMDENCITYNEWS,local,Welcome to the big house Take our quiz on high...


In [121]:
#targets = ['Right', 'Left', 'local','news','Russian','Hashtager','Commercial']
targets = ['Right', 'Left']
user_documents = user_documents.loc[user_documents.account_type.isin(targets)]

In [122]:
user_documents.head(10)

,author,account_type,content
0,10_GOP,Right,"""We have a sitting Democrat US Senator on tria..."
2,1ERIK_LEE,Right,Why is someone even against the I'll watch you...
4,2NDHALFONION,Right,' Follow the other half an onion Mage America ...
6,4EVER_SUSAN,Right,defense playing hungry Bending and not breaki...
7,4MYSQUAD,Left,' so If you pay more taxes you have more righ...
16,AANTIRACIST,Left,Black sergeant charged for ‘failure to supervi...
20,ABBYLOPTRT,Right,Newt Explains How Scaramucci Will Change the W...
21,ABELLABAB,Left,Ferguson residents were treated like slaves be...
24,ABIISSROSB,Right,Muslim Terror Group Defends Mc Master Fox Ne...
25,ABISADMASST,Right,Mike Huckabee Congratulates New Press Secretar...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=preprocess, ngram_range=(1,3)) 

tfidf_matrix = tfidf_vectorizer.fit_transform(user_documents.content) #fit the vectorizer to content

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()
terms[0:20]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(tfidf_matrix)
dist

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 2

km = KMeans(n_clusters=n_clusters)

km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [ ]:
user_documents['clusters'] = clusters

In [ ]:
user_documents['content'] = user_documents.apply(lambda row: preprocess(row['content'].lower()), axis=1)

In [ ]:
grouped = user_documents.groupby('account_type', as_index=False)['author','content','clusters'].agg({'author': list, 'clusters': lambda x: scipy.stats.mode(x)[0]})
grouped

In [ ]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(n_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for user in user_documents.ix[i]['author'].values.tolist():
        print(' %s,' % user, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()